Advanced: Built-In Constraints
============================

Setup
-----------------------------

Let's first make sure we have the latest version of PHOEBE 2.4 installed (uncomment this line if running in an online notebook session such as colab).

In [1]:
#!pip install -I "phoebe>=2.4,<2.5"

As always, let's do imports and initialize a logger and a new Bundle.

In [2]:
import phoebe
from phoebe import u # units
import numpy as np
import matplotlib.pyplot as plt

logger = phoebe.logger()

b = phoebe.default_binary()

Built-in Constraints
-----------------------------

There are a number of [built-in constraints](../api/phoebe.parameters.constraint.md) that can be applied to our system.  Those added by default are listed below as well as in the API docs for [b.add_constraint](../api/phoebe.frontend.bundle.Bundle.add_constraint.md):

### asini

These constraint handles computing the projected semi-major axis (either for an orbit or a star) along the line of sight and can be automatically inverted to solve for either 'asini', 'sma', or 'incl'.

In [3]:
b.filter(qualifier='asini', context='constraint')

<ParameterSet: 3 parameters | kinds: orbit, star>

In [4]:
b.get_parameter(qualifier='asini', component='binary', context='constraint')

<ConstraintParameter: {asini@binary@component} = {sma@binary@component} * (sin({incl@binary@component})) (solar units) => 5.3 solRad>

### esinw, ecosw

These constraints handle computing the projected eccentricity which can be helpful in that they are better representations of the *geometry* of a light curve and result in symmetric posteriors for near-circular orbits.

Both can be inverted to also automatically solve for 'ecc' or 'per0'.

In [5]:
b.get_parameter(qualifier='esinw', context='constraint')

<ConstraintParameter: {esinw@binary@component} = {ecc@binary@component} * (sin({per0@binary@component})) (solar units) => 0.0>

In [6]:
b.get_parameter(qualifier='ecosw', context='constraint')

<ConstraintParameter: {ecosw@binary@component} = {ecc@binary@component} * (cos({per0@binary@component})) (solar units) => 0.0>

### t0

This constraint handles converting between different t0 conventions - namely providing a reference time at periastron passage (t0_perpass) and at superior conjunction (t0_supconj).

Currently, this constraint only supports inverting to be solved for 't0_supconj' (ie you cannot *automatically* invert this constraint to constraint phshift or per0).

In [7]:
b.get_parameter(qualifier='t0_perpass', context='constraint')

<ConstraintParameter: {t0_perpass@binary@component} = t0_supconj_to_perpass({t0_supconj@binary@component}, {period@binary@component}, {ecc@binary@component}, {per0@binary@component}, {dpdt@binary@component}, {dperdt@binary@component}, {t0@system}) (solar units) => -0.25 d>

### freq

This constraint handles the simple conversion to frequency from period - whether that be rotational or orbital - and does support inversion to solve for 'period'.

In [8]:
b.filter(qualifier='freq', context='constraint')

<ParameterSet: 3 parameters | kinds: orbit, star>

In [9]:
b.get_parameter(qualifier='freq', component='binary', context='constraint')

<ConstraintParameter: {freq@binary@component} = 6.283185 / {period@binary@component} (solar units) => 6.283185 rad / d>

In [10]:
b.get_parameter(qualifier='freq', component='primary', context='constraint')

<ConstraintParameter: {freq@primary@component} = 6.283185 / {period@primary@component} (solar units) => 6.283185 rad / d>

### mass

This constraint handles solving for the mass of a component by obeying Kepler's third law within the parent orbit.

It can be inverted to solve for 'sma', 'q', or 'period' (in addition to 'mass').

In [11]:
b.filter(qualifier='mass', context='constraint')

<ParameterSet: 2 parameters | components: secondary, primary>

In [12]:
b.get_parameter(qualifier='mass', component='primary', context='constraint')

<ConstraintParameter: {mass@primary@component} = (39.478418 * ({sma@binary@component} ** 3.000000)) / ((({period@binary@component} ** 2.000000) * ({q@binary@component} + 1.000000)) * 2942.206217504419328179210424423218) (solar units) => 0.9988131358058301 solMass>

### component sma

This constraint handles computing the semi-major axis of a component about the **center of mass** of its parent orbit.  Note that this is **not** the same as the semi-major axis **of** the parent orbit.

This currently can be inverted to solve for 'sma' of the parent orbit, but **not** 'q'.

In [13]:
b.filter(qualifier='sma', context='constraint')

<ParameterSet: 2 parameters | components: secondary, primary>

In [14]:
b.get_parameter(qualifier='sma', component='primary', context='constraint')

<ConstraintParameter: {sma@primary@component} = {sma@binary@component} / ((1.000000 / {q@binary@component}) + 1.000000) (solar units) => 2.65 solRad>

### component asini

This constraint handles computing the projected semi-major axis of a component about the **center of mass** of its parent orbit.  Note that this is **not** the same as the asini **of** the parent orbit.

This currently can be inverted to solve for 'sma' of the parent orbit, but **not** 'q' or 'incl'.

In [15]:
b.filter(qualifier='asini', context='constraint')

<ParameterSet: 3 parameters | kinds: orbit, star>

In [16]:
b.get_parameter(qualifier='asini', component='primary', context='constraint')

<ConstraintParameter: {asini@primary@component} = ({sma@binary@component} * (sin({incl@binary@component}))) / ((1.000000 / {q@binary@component}) + 1.000000) (solar units) => 2.65 solRad>

### requiv_max

This constraint handles solving for the maxium equivalent radius (for a detached system).

For a [semi-detached system](./requiv_crit_semidetached.ipynb), the radius itself is constrained to be exactly this value.

In [17]:
b.filter(qualifier='requiv_max', context='constraint')

<ParameterSet: 2 parameters | components: secondary, primary>

In [18]:
b.get_parameter(qualifier='requiv_max', component='primary', context='constraint')

<ConstraintParameter: {requiv_max@primary@component} = requiv_L1({q@binary@component}, {syncpar@primary@component}, {ecc@binary@component}, {sma@binary@component}, {incl@primary@component}, {long_an@primary@component}, {incl@binary@component}, {long_an@binary@component}, 1) (solar units) => 2.013275176537638 solRad>

### rotation period

This constraint handles computing the rotation period of a star given its synchronicity parameter (syncpar).

It can be inverted to solve for any of the three parameters 'period' (both rotational and orbital) and 'syncpar'.

In [19]:
b.filter(qualifier='period', context='constraint')

<ParameterSet: 2 parameters | components: secondary, primary>

In [20]:
b.get_parameter(qualifier='period', component='primary', context='constraint')

<ConstraintParameter: {period@primary@component} = {period@binary@component} / {syncpar@primary@component} (solar units) => 1.0 d>

### pitch/yaw (incl/long_an)

pitch constrains the relation between the orbital and rotational inclination whereas yaw constrains the relation between the orbital and rotational long_an.  When pitch **and** yaw are set to 0, the system is aligned.

In [21]:
b.filter(qualifier='incl', context='constraint')

<ParameterSet: 2 parameters | components: secondary, primary>

In [22]:
b.get_parameter(qualifier='incl', component='primary', context='constraint')

<ConstraintParameter: {incl@primary@component} = {incl@binary@component} + {pitch@primary@component} (solar units) => 90.0 deg>

In [23]:
b.filter(qualifier='long_an', context='constraint')

<ParameterSet: 2 parameters | components: secondary, primary>

In [24]:
b.get_parameter(qualifier='long_an', component='primary', context='constraint')

<ConstraintParameter: {long_an@primary@component} = {long_an@binary@component} + {yaw@primary@component} (solar units) => 0.0 deg>